In [2]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
from urllib.parse import urlparse
import sqlite3,time
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

In [3]:
sql_data = 'database.db'
conn = sqlite3.connect(sql_data)
conn.row_factory = dict_factory

In [4]:
query ="""SELECT `javascript.enabled`, `document.referrer`, validity, `server.country`, `server.useragent.info`, `server.region`, `var.ClientID` FROM sample_sql_db"""

records = conn.execute(query).fetchall()


ids = list(set([x['var.ClientID'] for x in records ]))

print(ids)


['3blue']


In [18]:
def prepareDataframe(start_date, end_date, client_id='3blue'):

    print("Preparing DF for ", client_id)
    query ="""SELECT `javascript.enabled`, `document.referrer`, validity, `server.country`, `server.useragent.info`, `server.region`, count(id) as visits FROM sample_sql_db
    WHERE `pixel.timestamp` > {}
    AND `pixel.timestamp` < {}
    AND `var.ClientID` = '{}'
    GROUP BY `javascript.enabled`, `document.referrer`, validity, `server.country`, `server.useragent.info`, `server.region`
    ORDER BY visits DESC""".format(start_date, end_date, client_id)
    records = conn.execute(query).fetchall()

    columns = ['javascript.enabled','document.referrer','validity','server.country','server.useragent.info','server.region','visits']
    df = pd.DataFrame(data=[x.values() for x in records],columns=columns)

    df['document.referrer'] = df['document.referrer'].astype(str)
    df['javascript.enabled'] = df['javascript.enabled'].astype(str)
    return df

start_date = time.mktime(datetime.strptime("2022-04-01", "%Y-%m-%d").timetuple())
end_date = time.mktime(datetime.strptime("2022-04-25", "%Y-%m-%d").timetuple())
client_id = '3blue' 


df = prepareDataframe(start_date, end_date, client_id)


df.shape


Preparing DF for  3blue


(4477, 7)

In [26]:
list(set([x for x in list(set(df['document.referrer'].values.tolist())) ]))


# set([x for x in df['javascript.enabled']])
# set([x for x in df['document.referrer']])
# 

['https://fizzsy.com/monitordt3.php?i=zNT1_8280---8ea64ec7&url=https%3A%2F%2Ffizzsy.com%2Fnothing-improves-a-day-like-cheesy-garlic-bread%2F&sb_url=https%3A%2F%2Ffizzsy.com%2Fsidebar-elements-2%2F',
 'https://fizzsy.com/monitordt3.php?i=zbru1_4b44330b-8626-4d3b-97ac-3b9ece3cc143-futuresmodel.com&url=https%3A%2F%2Ffizzsy.com%2Fnothing-improves-a-day-like-cheesy-garlic-bread%2F&sb_url=https%3A%2F%2Ffizzsy.com%2Fsidebar-elements-2%2F',
 'http://syn.entertainow.com/feed?type=email&c=KHWOapSM1Cmzmiee&f=NalYczz99RoqOKgh&ch=88efefdd-a127-43d4-800b-f603b3258115&source=A1&uid=44146942&sig=029bea66a0bc121efc312b93e048ce7cd88f921e&ts=1649881574&clientSig=9fd1a182a030797750c58e6c344b5753',
 'https://fizzsy.com/monitordt3.php?i=zCat_1&url=https%3A%2F%2Ffizzsy.com%2Fnothing-improves-a-day-like-cheesy-garlic-bread%2F&sb_url=https%3A%2F%2Ffizzsy.com%2Fsidebar-elements-2%2F',
 'https://fizzsy.com/cooking-together/',
 'https://fizzsy.com/monitordt3.php?i=zbru1_7df60840-f94d-4692-acc3-2c864b7f390d-chess.

In [ ]:
# df = pd.read_csv("data.csv") 
df = pd.read_excel(open('data.xlsx','rb'), sheet_name='Sheet1')

# with pd.ExcelFile("data.xlsx") as reader:
#     df = pd.read_excel(reader, sheet_name='Sheet1')



In [53]:

def getBotSourceDataTable():
    alphabetical = ['Direct', 'Search', 'Social',  'Display' , 'Unknown']
    tbody = []
    local_df = df
    temp_df = local_df[(local_df['javascript.enabled'] == '1') |  (local_df['javascript.enabled'] == "true")]
    y =  temp_df[  temp_df["document.referrer"].str.contains("site-tracking.com|fisszy|")]
  
    print(y.shape)
    
    
    for index,source in enumerate(alphabetical[:]):
        
    #     # Direct
        if index==0:
            temp_df2 = temp_df[  (temp_df['document.referrer'] == 'None') ]
        elif index==1:
            temp_df2 = temp_df[  temp_df["document.referrer"].str.contains("baidu|google|bing|duckduckgo")    ]
        elif index==2:
            temp_df2 = temp_df[  temp_df["document.referrer"].str.contains("baidu|google|facebook|t.co")    ]
        elif index==3:
            temp_df2 = temp_df[  temp_df["document.referrer"].str.contains("site-tracking.com|fisszy")    ]
            
        elif index==4:
            temp_df2 = local_df[  (local_df['javascript.enabled'] == 'None') ]
       
       
            
            
            
            
        total_rows = temp_df2['visits'].sum()
        if total_rows>0:
            humans = round(temp_df2[(temp_df2['validity']=='valid')]['visits'].sum() / total_rows * 100,2)
            bots = round(temp_df2[(temp_df2['validity']=='invalid') | (temp_df2['validity']=='suspicious')]['visits'].sum() / total_rows * 100,2)
            unknown = round(temp_df2[(temp_df2['validity']=='unknown')]['visits'].sum() / total_rows * 100,2)
        else:
            humans = 0
            bots = 0
            unknown = 0
            
        tbody.append([
            source, total_rows , humans, bots , unknown
        ])
    tbody = sorted(tbody, key=lambda x: x[1])[::-1]
    
    return {
        "thead": ["Bot Source", "Site Visitors", "% Humans", "% Bots"],
        "tbody": tbody,
        "filters": alphabetical,
        "dropdown_id":"bot_sources_dropdown",
        "table_id":"bot_sources",
        "title":"Bot Sources",
        
    } 
            
            
    
    

res = getBotSourceDataTable()



res

(0, 7)


{'thead': ['Bot Source', 'Site Visitors', '% Humans', '% Bots'],
 'tbody': [['Unknown', 32347, 0.0, 8.39, 91.61],
  ['Direct', 14605, 88.24, 11.76, 0.0],
  ['Social', 2, 100.0, 0.0, 0.0],
  ['Display', 0, 0, 0, 0],
  ['Search', 0, 0, 0, 0]],
 'filters': ['Direct', 'Search', 'Social', 'Display', 'Unknown'],
 'dropdown_id': 'bot_sources_dropdown',
 'table_id': 'bot_sources',
 'title': 'Bot Sources'}

In [19]:
def getDomainDataTabel(): 
    alphabetical =  list(set(df['document.referrer'].values.tolist()))
    alphabetical = [x for x in alphabetical if x]
    alphabetical = list(set([urlparse(x).netloc.replace(".com","").replace("www.","").capitalize() for x in alphabetical]))
    alphabetical = [x for x in alphabetical if x] 
    
    tbody = []
    for index,source in enumerate(alphabetical[:]):
        temp_df = df[  df["document.referrer"].str.contains(source.lower(), na=False)   ]
        
        total_rows = temp_df['visits'].sum()
        if total_rows>0:
            humans = round(temp_df[(temp_df['validity']=='valid')]['visits'].sum() / total_rows * 100,2)
            bots = round(temp_df[(temp_df['validity']=='invalid') | (temp_df['validity']=='suspicious')]['visits'].sum() / total_rows * 100,2)
            unknown = round(temp_df[(temp_df['validity']=='unknown')]['visits'].sum() / total_rows * 100,2)
        else:
            humans = 0
            bots = 0
            unknown = 0
            
        tbody.append([
            source, total_rows , humans, bots , unknown
        ])
        
    tbody = sorted(tbody, key=lambda x: x[1])[::-1]
    return {
        "thead": ["Bot Source", "Site Visitors", "% Humans", "% Bots"],
        "tbody": tbody,
        "filters": alphabetical,
        "dropdown_id":"domain_performance_dropdown",
        "title":"Domain / Platform Reporting",
        "table_id":"domain_performance",
        
    } 
    
getDomainDataTabel()

{'thead': ['Bot Source', 'Site Visitors', '% Humans', '% Bots'],
 'tbody': [['Fizzsy', 83549, 88.89, 11.11, 0.0],
  ['Syn.entertainow', 1, 0.0, 100.0, 0.0]],
 'filters': ['Fizzsy', 'Syn.entertainow'],
 'dropdown_id': 'domain_performance_dropdown',
 'title': 'Domain / Platform Reporting',
 'table_id': 'domain_performance'}

In [11]:
def getCountryDataTable():
    alphabetical =  list(set(df['server.country'].values.tolist()))
    alphabetical = [x for x in alphabetical if not str(x)=='[NULL]'  and type(x) is str] 
    print(alphabetical)
    tbody = []
    for index,source in enumerate(alphabetical[:]):
        temp_df = df[  (df['server.country'] ==  str(source) ) ]
        total_rows = temp_df['visits'].sum() 
        
        if total_rows>0:
            humans = round(temp_df[(temp_df['validity']=='valid')]['visits'].sum() / total_rows * 100,2)
            bots = round(temp_df[(temp_df['validity']=='invalid') | (temp_df['validity']=='suspicious')]['visits'].sum() / total_rows * 100,2)
            unknown = round(temp_df[(temp_df['validity']=='unknown')]['visits'].sum() / total_rows * 100,2)
        else:
            humans = 0
            bots = 0
            unknown = 0
        
        if type(source) is str: 
            tbody.append([
                source, total_rows , humans, bots , unknown, {"show":True}
            ])
         
        
    tbody = sorted(tbody, key=lambda x: x[1])[::-1]
    return {
        "thead": ["Bot Source", "Site Visitors", "% Humans", "% Bots"],
        "tbody": tbody,
        "filters": alphabetical,
        "dropdown_id":"country_data_table_dropdown",
        "title":"Country Bots",
        "table_id":"country_data_table",
    } 
    
res = getCountryDataTable()

res

[]


{'thead': ['Bot Source', 'Site Visitors', '% Humans', '% Bots'],
 'tbody': [],
 'filters': [],
 'dropdown_id': 'country_data_table_dropdown',
 'title': 'Country Bots',
 'table_id': 'country_data_table'}

In [93]:
def getStatBarData():
    temp_df = df 
    total_site_users = temp_df['visits'].sum()
    humans = round(temp_df[(temp_df['validity']=='valid')]['visits'].sum()/total_site_users*100,2  ) 
    print("Valid = ", temp_df[(temp_df['validity']=='valid')]['visits'].sum())
    invalid_bots = temp_df[temp_df["validity"].str.contains("invalid")]['visits'].sum()
    total_bots = temp_df[temp_df["validity"].str.contains("invalid")]['visits'].sum()
    friendly_bots = temp_df[temp_df["server.useragent.info"].str.contains("bot") & temp_df["server.useragent.info"].str.contains("Googlebot|BingBot|yandex|archive.org") ].shape[0]
    friendly_bots = round(friendly_bots/total_bots*100 ,2)
    malicious_bots = invalid_bots - friendly_bots
    malicious_bots = round(malicious_bots/total_bots*100,2)
    bots = round(total_bots/total_site_users*100,2)

    
    print("bots - ",bots )
    print("malicious_bots - ",malicious_bots )
    print("friendly_bots - ",friendly_bots )
    
    return {
        "total_site_users":total_site_users,
        "humans":humans,
        "friendly_bots":friendly_bots,
        "malicious_bots":malicious_bots,
        "bots":bots,
    }
    
    
 
    
getStatBarData()

Valid =  5831
bots -  4.24
malicious_bots -  99.91
friendly_bots -  0.29


{'total_site_users': 8000,
 'humans': 72.89,
 'friendly_bots': 0.29,
 'malicious_bots': 99.91,
 'bots': 4.24}

In [88]:
def getMapData():
    temp_df = df[  (df['server.country'] == 'United States') ]
    total_rows = temp_df['visits'].sum()
 
    states = temp_df['server.region'].values.tolist()
    states = list(set([x for x in states if str(x) not in  ['[NULL]', 'nan','NULL','NONE'] ]))
    states = [x for x in states if x]
    
    
    stateDictionary =  {"Alaska" : "AK", "Alabama" : "AL", "Arkansas" : "AR", "American Samoa" : "AS", "Arizona" : "AZ", "California" : "CA", "Colorado" : "CO", "Connecticut" : "CT", "District of Columbia" : "DC", "Delaware" : "DE", "Florida" : "FL", "Georgia" : "GA", "Guam" : "GU", "Hawaii" : "HI", "Iowa" : "IA", "Idaho" : "ID", "Illinois" : "IL", "Indiana" : "IN", "Kansas" : "KS", "Kentucky" : "KY", "Louisiana" : "LA", "Massachusetts" : "MA", "Maryland" : "MD", "Maine" : "ME", "Michigan" : "MI", "Minnesota" : "MN", "Missouri" : "MO", "Mississippi" : "MS", "Montana" : "MT", "North Carolina" : "NC", "North Dakota" : "ND", "Nebraska" : "NE", "New Hampshire" : "NH", "New Jersey" : "NJ", "New Mexico" : "NM", "Nevada" : "NV", "New York" : "NY", "Ohio" : "OH", "Oklahoma" : "OK", "Oregon" : "OR", "Pennsylvania" : "PA", "Puerto Rico" : "PR", "Rhode Island" : "RI", "South Carolina" : "SC", "South Dakota" : "SD", "Tennessee" : "TN", "Texas" : "TX", "Utah" : "UT", "Virginia" : "VA", "Virgin Islands" : "VI", "Vermont" : "VT", "Washington" : "WA", "Wisconsin" : "WI", "West Virginia" : "WV", "Wyoming" : "WY"}
    
    density = []
    total_site_users_list = []
    percentage_bots = []
    percentage_humans = []
    percentage_friendly_bots = []
    percentage_malicious_bots = []
    for state in states:
        local = df[(df['server.region'] ==state)]
        bots = local[local["validity"].str.contains("invalid")]['visits'].sum()
        
        if bots>0:
            bots = round(bots/local['visits'].sum() *100,2)
        density.append(bots)    
        temp_df2 = df[  (df['server.region'] == state) ] 
        total_site_users = temp_df2['visits'].sum() 
        if total_site_users>0:
            humans = round(temp_df2[(temp_df2['validity']=='valid')]['visits'].sum()/total_site_users*100,2  ) 
        else:
            humans=0
        invalid_bots = temp_df2[temp_df2["validity"].str.contains("invalid")]['visits'].sum()
        total_bots = temp_df2[temp_df2["validity"].str.contains("invalid")]['visits'].sum() 
        friendly_bots = temp_df2[temp_df2["server.useragent.info"].str.contains("bot") & temp_df2["server.useragent.info"].str.contains("Googlebot|BingBot|yandex|archive.org") ].shape[0]
        if total_bots>0:
            friendly_bots = round(friendly_bots/total_bots*100 ,2)
            malicious_bots = invalid_bots - friendly_bots
            malicious_bots = round(malicious_bots/total_bots*100,2)
            bots = round(total_bots/total_site_users*100,2)
        else:
            friendly_bots = 0
            malicious_bots = 0
            malicious_bots = 0
            bots = 0
        
        total_site_users_list.append(total_site_users)
        percentage_bots.append(bots)
        percentage_humans.append(humans)
        percentage_friendly_bots.append(friendly_bots)
        percentage_malicious_bots.append(malicious_bots)
        
        
    states = [stateDictionary[x] for x in states]
    density = [100 if x>10 else 0 for x in density]
    percentage_malicious_bots = [x if x>0 else 0 for x in percentage_malicious_bots]
    
    columns = ['states','density','total_site_users','percentage_bots','percentage_humans','percentage_friendly_bots','percentage_malicious_bots']
    return pd.DataFrame(data=zip(states,density,total_site_users_list,percentage_bots,percentage_humans,percentage_friendly_bots,percentage_malicious_bots), columns=columns) 

res = getMapData()
 